# LangChain 实战：汽车销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chatgpt.com/share/280e7f19-8ceb-4897-92d2-261ea61c7b48


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国汽车行业内著名的销售，你不仅拥有丰富的汽车知识，还能很准确的拿捏客户的心理现在你在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
1.
[客户问题] 这车的安全性能怎么样？
[销售回答] 当然了，这款车配备了最新的安全系统，包括主动刹车、车道保持和多气囊设计，确保您的驾驶安全。

```

GPT-4 回复结果：

```
1.
[客户问题] 这车的安全性能怎么样？
[销售回答] 当然了，这款车配备了最新的安全系统，包括主动刹车、车道保持和多气囊设计，确保您的驾驶安全。

2.
[客户问题] 这车的油耗高吗？
[销售回答] 不高的，这款车采用了先进的燃油经济性技术，市区和高速公路上的综合油耗都非常低，省油又环保。

3.
[客户问题] 这车的保值率怎么样？
[销售回答] 很高，这款车一直以来保值率都非常高，因为它的品质可靠，市场认可度也很高，未来转手的时候一定能卖个好价钱。

4.
[客户问题] 车子的售后服务怎么样？
[销售回答] 非常完善，我们提供全方位的售后服务，不仅有全国联保，还有24小时道路救援，确保您用车无忧。

5.
[客户问题] 这个品牌的车子质量可靠吗？
[销售回答] 当然，我们这个品牌在市场上已经有多年历史，口碑非常好，质量经过市场的长期检验，您可以完全放心。

6.
[客户问题] 这车的驾驶感受怎么样？
[销售回答] 非常出色，这款车的悬挂系统和动力系统经过精心调校，驾驶起来非常平稳舒适，而且转向精准，给您带来愉悦的驾驶体验。

7.
[客户问题] 这车的维修成本高吗？
[销售回答] 不高的，我们的车子使用了高质量的零部件，耐用性强，日常保养和维修成本相对较低，性价比非常高。

8.
[客户问题] 这车的内饰怎么样？
[销售回答] 很豪华，这款车的内饰采用了高档材质，设计风格简约时尚，坐进去您会感到非常舒适和豪华。

9.
[客户问题] 这车的动力表现如何？
[销售回答] 非常强劲，我们这款车搭载了高效发动机，动力输出强劲，不论是城市驾驶还是高速行驶，都能给您带来畅快的体验。

10.
[客户问题] 这车的配置怎么样？
[销售回答] 非常丰富，这款车的配置非常丰富，配备了最新的智能科技和舒适性配置，让您的驾驶更轻松愉快。

```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [8]:
with open("real_estate_sales_data.txt", encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [9]:
from langchain.text_splitter import CharacterTextSplitter

In [10]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [11]:
docs = text_splitter.create_documents([real_estate_sales])

In [12]:
docs[0]

Document(page_content='[客户问题] 这车的安全性能怎么样？\n[销售回答] 当然了，这款车配备了最新的安全系统，包括主动刹车、车道保持和多气囊设计，确保您的驾驶安全。')

In [13]:
len(docs)

100

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [14]:
import os
os.environ["OPENAI_API_KEY"] = 'sk-k9jM0trGFbAQm1cu6bD951Cd503c4203A64d6aFf7aD8Ff09'
os.environ["OPENAI_BASE_URL"] = 'https://api.xiaoai.plus/v1'

In [15]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [16]:
query = "这车的内饰设计风格如何？"

In [17]:
answer_list = db.similarity_search(query)

In [18]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 这车的内饰设计风格如何？
[销售回答] 非常时尚，这款车的内饰设计风格现代简约，每一个细节都经过精心设计，营造出高档的车内氛围。

[客户问题] 这车的内饰怎么样？
[销售回答] 很豪华，这款车的内饰采用了高档材质，设计风格简约时尚，坐进去您会感到非常舒适和豪华。

[客户问题] 这车的外观设计如何？
[销售回答] 非常时尚，这款车的外观由知名设计师操刀，线条流畅、造型时尚，开出去绝对能吸引眼球。

[客户问题] 这车的内饰细节做工如何？
[销售回答] 非常精致，这款车的内饰细节做工非常考究，每一个细节都体现了高档品质，让您感受到真正的豪华。



In [19]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [20]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [21]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000279F397B550>, search_kwargs={'k': 3})

In [22]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 这车的内饰设计风格如何？
[销售回答] 非常时尚，这款车的内饰设计风格现代简约，每一个细节都经过精心设计，营造出高档的车内氛围。

[客户问题] 这车的内饰怎么样？
[销售回答] 很豪华，这款车的内饰采用了高档材质，设计风格简约时尚，坐进去您会感到非常舒适和豪华。

[客户问题] 这车的外观设计如何？
[销售回答] 非常时尚，这款车的外观由知名设计师操刀，线条流畅、造型时尚，开出去绝对能吸引眼球。



In [23]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪车啊？")

In [24]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这车的内饰怎么样？
[销售回答] 很豪华，这款车的内饰采用了高档材质，设计风格简约时尚，坐进去您会感到非常舒适和豪华。

[客户问题] 这车的颜色选择多吗？
[销售回答] 非常多，这款车提供了多种颜色选择，从经典的黑白灰到时尚的亮色，总有一款适合您的喜好。

[客户问题] 这车的贷款方案怎么样？
[销售回答] 很灵活，我们提供多种灵活的贷款方案，首付比例和还款期限都可以根据您的需求调整，帮助您轻松购车。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [25]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [26]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这车的内饰设计风格如何？
[销售回答] 非常时尚，这款车的内饰设计风格现代简约，每一个细节都经过精心设计，营造出高档的车内氛围。

[客户问题] 这车的内饰怎么样？
[销售回答] 很豪华，这款车的内饰采用了高档材质，设计风格简约时尚，坐进去您会感到非常舒适和豪华。

[客户问题] 这车的外观设计如何？
[销售回答] 非常时尚，这款车的外观由知名设计师操刀，线条流畅、造型时尚，开出去绝对能吸引眼球。

[客户问题] 这车的内饰细节做工如何？
[销售回答] 非常精致，这款车的内饰细节做工非常考究，每一个细节都体现了高档品质，让您感受到真正的豪华。



### 提取向量数据库中的`销售回答`

In [27]:
docs = retriever.get_relevant_documents(query)

In [28]:
docs[0].page_content

'[客户问题] 这车的内饰设计风格如何？\n[销售回答] 非常时尚，这款车的内饰设计风格现代简约，每一个细节都经过精心设计，营造出高档的车内氛围。'

In [29]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 这车的内饰设计风格如何？\n', '非常时尚，这款车的内饰设计风格现代简约，每一个细节都经过精心设计，营造出高档的车内氛围。']

In [30]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [31]:
ans

'非常时尚，这款车的内饰设计风格现代简约，每一个细节都经过精心设计，营造出高档的车内氛围。'

#### 尝试各种问题

In [32]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [33]:
query = "我想要提速快点的"

print(sales(query))

[]


D:\miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [34]:
print(sales(query, 0.75))

[]


D:\miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


In [35]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



D:\miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['很高，这款车一直以来保值率都非常高，因为它的品质可靠，市场认可度也很高，未来转手的时候一定能卖个好价钱。', '很豪华，这款车的内饰采用了高档材质，设计风格简约时尚，坐进去您会感到非常舒适和豪华。', '不高的，我们与多家保险公司有合作，可以为您提供优惠的保险方案，费用相对较低，保障全面。', '非常精致，这款车的内饰细节做工非常考究，每一个细节都体现了高档品质，让您感受到真正的豪华。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [36]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [37]:
qa_chain({"query": "你们有200万的豪车吗？"})

D:\miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
D:\miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们有200万的豪车吗？',
 'result': '对不起，我不知道。我只能提供信息和帮助回答问题，但我无法销售商品或服务。如果你有兴趣购买豪车，建议你联系当地的汽车经销商。'}

In [38]:
qa_chain({"query": "这车的安全性能怎么样？"})

{'query': '这车的安全性能怎么样？', 'result': '这款车配备了最新的安全系统，包括主动刹车、车道保持和多气囊设计，确保您的驾驶安全。'}

In [39]:
print(sales("这车的安全性能怎么样？"))

['当然了，这款车配备了最新的安全系统，包括主动刹车、车道保持和多气囊设计，确保您的驾驶安全。', '非常安全，这款车的车身结构采用了高强度钢材，碰撞测试成绩非常优秀，确保了乘员的安全。', '非常好，这款车配备了高性能轮胎，抓地力强、耐磨性好，保证了行车的稳定性和安全性。', '很优秀，这款车的操控性能非常出色，转向灵活、刹车灵敏，让您在驾驶过程中充满信心。']


## 加载 FAISS 向量数据库已有结果

In [40]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings())

ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).

In [41]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

D:\miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [42]:
qa_chain({"query": "我想买劳斯莱斯，你们有么"})

D:\miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买劳斯莱斯，你们有么',
 'result': '对不起，我不能帮助您购买劳斯莱斯，因为我是一个人工智能，无法进行实物交易。建议您联系汽车销售商或直接访问劳斯莱斯的官方网站进行查询。'}

In [43]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [38]:
qa_chain({"query": "我想买劳斯莱斯，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅，因为我是一个人工智能，没有提供房地产服务的功能。'}

In [39]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [40]:
result = qa_chain({"query": "我想买劳斯莱斯，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [41]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}